# Clusterização (Modelo Não Supervisionado K-means)
Este notebook explora o algoritmo de aprendizado de máquina não supervisionado K-Means. <br>
O objetivo deste método é agrupar dados em diferentes grupos (clusters), sem o uso de rótulos prévios, e buscar tendências. </p>

---
## Instalação de bibliotecas
Instala bibliotecas necessárias:

In [ ]:
!pip install pandas==2.1.4 numpy==1.26.4 matplotlib==3.7.5 seaborn==0.13.2 scikit-learn==1.4.2

---
## Inicialização de bibliotecas
Importa as bibliotecas necessárias:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA


# Configuração para mostrar todas as colunas no pandas
pd.set_option('display.max_columns', None)

# Configuração para exibir os gráficos diretamente no notebook
%matplotlib inline

---
## Importe do banco de dados via arquivo local
Importação do DataFrame da base de dados da Unipar com a biblioteca pandas:

Para rodar o notebook corretamente, será necessário importar o arquivo de banco de dados manualmente, pois ele não está incluído no repositório devido à presença de dados sensíveis.

#### Passos para Importar o Banco de Dados no VS Code

1. **Obtenha o Arquivo de Dados**:
   - O arquivo `BASE DE SINISTRO UNIPAR BRADESCO.csv` deverá ser fornecido separadamente. Entre em contato com o responsável pelo projeto para receber o arquivo.

2. **Posicione o Arquivo na Pasta Correta**:
   - Após receber o arquivo, arraste-o para a mesma pasta onde o notebook está localizado em seu computador. Isso garante que o caminho relativo na função de leitura permaneça o mesmo e funcione corretamente.
   
3. **Verifique o Caminho do Arquivo**:
   - O código de leitura do arquivo já está implementado no notebook e não precisa ser alterado:
     ```python
     df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')
     ```
   - Certifique-se de que o arquivo CSV esteja no mesmo diretório que o notebook para evitar problemas de caminho.

4. **Rodar o Notebook**:
   - Com o arquivo posicionado corretamente, execute o notebook normalmente. O pandas irá carregar os dados e você poderá seguir com a análise.

**Nota**: Caso o arquivo não esteja na mesma pasta que o notebook, o código não será capaz de localizar o banco de dados, resultando em um erro. Portanto, é essencial que o arquivo seja arrastado para o diretório correto antes da execução.

&ensp;Agora que você baixou os notebooks e o banco de dados, pode seguir para os próximos passos de execução, seja localmente ou no Google Colab.

In [3]:
df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')

---
## Pré-Processamento
Como apresentado no notebook 'Pré-Processamento', as linhas a seguir executam várias etapas para limpar e organizar o banco de dados.<br>
Os comentários no código explicam cada ação.

In [4]:
# Limpeza de dados

# Tratamento de valores nulos
df = df.dropna()
# Correção do ponto faltante em 'UNIPAR INDUPA DO BRASIL S.A'
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})
# Remoção de AGREGADO e DEPENDENTE
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]
df = df.drop(df_remove_d.index)
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]
df = df.drop(df_remove_a.index)
# Tratamento de valores duplicados
df = df.drop_duplicates(keep='last')

df.head(10)

df_base = df.copy()

---
## Codificação
Este trecho de código realiza a codificação de colunas categóricas, transformando-as em valores numéricos para facilitar o uso no modelo<br>
de machine learning. As colunas categóricas são mapeadas para índices numéricos, e o resultado é armazenado em um novo DataFrame.<br>
Além disso, a coluna de data é convertida para o formato 'YYYYMMDD', e as colunas numéricas e de data são adicionadas ao DataFrame final.

In [5]:
# Lista de colunas que serão codificadas (colunas categóricas)
colunas_para_codificar = [
    'Codigo Empresa Sinistro',
    'Sexo Sinistro',
    'Faixa-Etária Nova Sinistro',
    'Descricao Plano Sinistro',
    'Codigo Servico Sinistro',
]

# Colunas numéricas e de data que não precisam de codificação
colunas_nao_codificadas = [
    'Dt Data Sinistro',
    'Valor Pago Sinistro',
]

# Para armazenar as novas colunas codificadas
novas_colunas_codificadas = {}

# Codificação das colunas categóricas
for coluna in colunas_para_codificar:
        unique_sorted_values = sorted(df[coluna].unique())
        df[f'Codificada {coluna}'] = df[coluna].apply(lambda x: unique_sorted_values.index(x))
        # Armazenar as novas colunas codificadas
        novas_colunas_codificadas[f'Codificada {coluna}'] = df[f'Codificada {coluna}']

# Criando um novo DataFrame com as novas colunas codificadas
df_novo = pd.DataFrame(novas_colunas_codificadas)

# Convertendo a coluna 'Dt Data Sinistro' para o formato 'YYYYMMDD' (Ano-Mês-Dia)
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

# Adicionando as colunas numéricas e de data ao DataFrame final
for coluna in colunas_nao_codificadas:
    df_novo[coluna] = df[coluna]

df = df_novo

#### Verificação

In [ ]:
# Exibindo as 10 primeiras linhas do novo DataFrame
df.head(10).sort_values(by='Valor Pago Sinistro', ascending=False)

---
## Detecção e Remoção de Outliers
Esse bloco de código remove os outlier da coluna *Valor Pago Sinistro* aplicando o _IsolationForest_ com uma contaminação de 5%, criando um novo DataFrame chamado de `df_clean`. Além disso, ele padroniza as colunas numéricas e as armazena em uma nova variável `numeric_colums` apenas com as colunas numéricas do DataFrame para futuras análises.

In [ ]:
# # Aplicar IsolationForest para detectar outliers
# iso_forest = IsolationForest(contamination=0.05)  # 5% de contaminação, ajuste conforme necessário
# outliers = iso_forest.fit_predict(df[['Valor Pago Sinistro']])

# # Remover os outliers
# df_clean = df[outliers == 1]

# Corrige a chamada de df
df_clean = df

# Selecionar apenas as colunas numéricas para padronização
numeric_columns = df_clean.select_dtypes(include=['float64', 'int64']).columns

# Padronizar os dados
scaler = StandardScaler()
df_clean[numeric_columns] = scaler.fit_transform(df_clean[numeric_columns])

# Retorna a notação de df
df = df_clean

df

#### Verificação

In [ ]:
df_clean.describe().round(2) #Arredonda para duas casas decimais

---
## Tratamento de Dados
A primeira linha desse bloco converte a coluna **Dt Data Sinistro** para um formato de ano-mês-dia, separando-os em 3 componentes diferentes, porém ainda na mesma coluna.<br>
As próximas três linhas separam esses componentes em 3 colunas separadas.<br>
Após isso, realizamos a padronização dessas 3 colunas e criamos novas colunas, `Ano_scaled`, `Mes_scaled` e `Dia_scaled`, para armazenar a versão escalonada das colunas originais.


In [ ]:
# Converter a coluna para datetime
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%Y%m%d')

# Extrair componentes de ano, mês e dia
df['Ano'] = df['Dt Data Sinistro'].dt.year
df['Mes'] = df['Dt Data Sinistro'].dt.month
df['Dia'] = df['Dt Data Sinistro'].dt.day

# Padronizar (escalonar) os componentes se necessário
scaler = StandardScaler()

df[['Ano_scaled', 'Mes_scaled', 'Dia_scaled']] = scaler.fit_transform(df[['Ano', 'Mes', 'Dia']])

df

---
## Remoção de Colunas
Este código remove as colunas `Mes` e `Dt Data Sinistro` do DataFrame `df`. A remoção dessas colunas é feita diretamente no DataFrame, utilizando o parâmetro `inplace=True`, o que garante que as alterações sejam aplicadas sem a necessidade de criar uma cópia do DataFrame. O resultado é um DataFrame que não contém mais as colunas especificadas.

In [ ]:
df.drop(columns=['Ano', 'Mes', 'Dia'], inplace=True)

df.drop(columns=['Dt Data Sinistro'], inplace=True)

df.head()

#### Verificação

In [ ]:
df_clean.head()

---
## Preparação do Modelo
### Valor de K
O valor de K no K-means representa o número de clusters que o algoritmo tentará formar, agrupando dados semelhantes.<br>
É importante definir K adequadamente, pois ele determina quantos grupos distintos serão gerados no conjunto de dados.

### Normalização
Antes de calcular o valor de K propriamente dito, buscamos normalizar os dados para garantir que todas as variáveis<br>
tenham a mesma escala, evitando que variáveis com valores maiores influenciem mais o agrupamento do que as menores.<br>
Para isso, utilizamos a classe ``StandardScaler`` da biblioteca ``scikit-learn`` (sklearn).

In [12]:
scaler = StandardScaler()

dados_normalizado = scaler.fit_transform(df)

### *Elbow Method* (ou Método do Cotovelo)
Com os dados normalizados, o *Elbow Method* ajuda a determinar o número ideal de *clusters* (K) no K-means. Ele traça<br>
a inércia (soma das distâncias dos pontos aos seus centróides) para diferentes valores de K. O ponto onde a redução<br>
na inércia começa a diminuir significativamente forma um "cotovelo" no gráfico, sugerindo o valor ideal de K.

In [ ]:
k_values = range(1, 13)
wcss = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(8,5))
plt.plot(k_values, wcss, 'bo-', markersize=8)
plt.xlabel('Número de clusters (k)')
plt.ylabel('WCSS')
plt.title('Método Elbow para Encontrar o k Ideal')
plt.show()

---
## Modelo K-means
Após normalizar os dados e identificar o valor ideal de K usando o *Elbow Method* (e o cálculo do Silhouette Score),<br>
a próxima etapa é aplicar o K-means. Nessa fase, o algoritmo é inicializado com o valor de K escolhido.

### Inicialização do modelo
O código abaixo divide os dados em treino e teste, normaliza ambos, aplica o K-means com K=7 aos dados de treino e<br>
prevê clusters para os dados de teste. Finalmente, cria um DataFrame com os dados de teste e adiciona os rótulos<br>
 dos clusters atribuídos.

In [14]:
dados_normalizado = scaler.fit_transform(df)

k = 7
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(df)
df['cluster'] = clusters

### Quantidade de Elementos / cluster
Verificação de quantos dados estão atribuídos a cada cluster após a aplicação do K-means.

In [ ]:
pd.Series(df['cluster']).value_counts()

### Dados de um cluster específico
Verificação de quais dados pertencem a um cluster específico.

In [ ]:
# Filtrar os dados que pertencem ao cluster 1
dados_cluster_1 = df[df['cluster'] == 1]

# Verificar os dados do cluster 1
dados_cluster_1

### Silhouette Score
Por fim, calculamos o Silhouette Score que mede a qualidade dos clusters, avaliando a coesão interna e a separação<br>
 entre clusters, com valores próximos de 1 indicando uma boa separação.

In [ ]:
silhouette_avg = silhouette_score(df, df['cluster'])

print(f"Silhouette Score: {silhouette_avg}")

### Análise de clusters segundo Silhouette Score

#### **ATENÇÃO!!!**
**O código está comentado devido ao tempo de execução prolongado (pode chegar a horas dependendo do computador).**

In [18]:
# DESABILIDATO POR DEMORAR MUITO

# # Lista para armazenar os Silhouette Scores
# silhouette_scores = []

# # Variando k de 2 a 12
# for k in range(2, 13):
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(dados_normalizado)
    
#     # Prediz os clusters para os dados de teste
#     clusters_teste = kmeans.predict(dados_normalizado)
    
#     # Calcula o Silhouette Score
#     score = silhouette_score(dados_normalizado, clusters_teste)
#     silhouette_scores.append((k, score))
    
#     # print(f'k = {k}, Silhouette Score = {score}') # Verificar os resultados

# # Converte os resultados para um DataFrame para melhor visualização
# silhouette_scores_df = pd.DataFrame(silhouette_scores, columns=['k', 'Silhouette Score'])

# # Plotando o gráfico
# plt.figure(figsize=(10, 6))
# plt.plot(silhouette_scores_df['k'], silhouette_scores_df['Silhouette Score'], marker='o', linestyle='-', color='b')
# plt.title('Silhouette Score para Diferentes Valores de k')
# plt.xlabel('Número de clusters (k)')
# plt.ylabel('Silhouette Score')
# plt.xticks(range(2, 13))
# plt.show()

### Análise de Hiperparâmetros com GridSearchCV

In [ ]:
def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    # Evitar erro se apenas 1 cluster for formado
    if len(np.unique(labels)) > 1:
        return silhouette_score(X, labels)
    else:
        return -1

# Definindo o grid de hiperparâmetros
param_grid = {
    'n_clusters': [7],
    'init': ['k-means++'],
    'max_iter': [500]
}

# # Definindo o grid de hiperparâmetros para testar
# param_grid = {
#     'n_clusters': [3, 5, 7, 9],
#     'init': ['k-means++', 'random'],
#     'max_iter': [300, 500, 1000]
# }

# Configurando o GridSearchCV
grid_search = GridSearchCV(
    estimator=KMeans(random_state=42), 
    param_grid=param_grid, 
    scoring=silhouette_scorer, 
    cv=5,  # Cross-validation
    verbose=3,
)

# Executando o GridSearch
grid_search.fit(dados_normalizado)

# Exibindo os melhores parâmetros encontrados
print("Melhores parâmetros: ", grid_search.best_params_)
print("Melhor silhouette score: ", grid_search.best_score_)

In [20]:
# sns.pairplot(df)

### Visualização dos clusters

In [ ]:
# Parâmetros ótimos para df: {'init': 'random', 'max_iter': 500, 'n_clusters': 7}
X = df.drop('cluster', axis=1)

# Visualizando os resultados com os melhores parâmetros
best_model = grid_search.best_estimator_
df['cluster'] = best_model.predict(X)

# Visualizando os clusters
scatter = plt.scatter(df['Mes_scaled'],df['Codificada Codigo Servico Sinistro'], c=df['cluster'], cmap='Spectral', edgecolors='black', linewidths=0.2)
# plt.scatter(cntr[:, 0], cntr[:, 1], c='red', marker='x')  # Centros dos clusters
plt.title('K-Means clustering com Melhores Parâmetros')
plt.xlabel('Valor Pago Sinistro')
plt.ylabel('Codificada Codigo Servico Sinistro')

# Lista de números de clusters
clusters = np.unique(df['cluster'])

# Criando a legenda com base nos clusters
legend_labels = [f'cluster {int(cluster)+1}' for cluster in clusters]

# Criando a barra de cores para o gráfico
cbar = plt.colorbar(scatter)
cbar.set_ticks(clusters)
cbar.set_ticklabels(legend_labels)

plt.show()

### Análise Gráfica
Após a clusterização, essa seção visa analisar graficamente os clusters resultantes do K-means e suas características.<br>
Para isso, criamos gráficos associando os clusters e as features que escolhemos de entrada para aplicar o modelo.

#### Distribuição de Elementos por cluster
Esse gráfico mostra a quantidade de elementos em cada cluster, permitindo visualizar a distribuição e a densidade<br>
dos dados entre os clusters.

In [ ]:
# Contar o número de observações em cada cluster
cluster_counts = df['cluster'].value_counts().sort_index()

# Criar um DataFrame para a visualização
cluster_counts_df = pd.DataFrame({'cluster': cluster_counts.index, 'Count': cluster_counts.values})

# Plotar o gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x='cluster', y='Count', hue='cluster', data=cluster_counts_df, palette='viridis', legend=False)
plt.title('Distribuição de Elementos por cluster')
plt.xlabel('cluster')
plt.ylabel('Número de Observações')
plt.xticks(rotation=45)  # Opcional: Rotacionar os rótulos do eixo x se necessário
plt.tight_layout()
plt.show()

#### Relação entre Homens e Mulheres por cluster
Esse gráfico mostra a distribuição de gêneros em cada cluster, permitindo visualizar como os grupos se dividem<br>
entre homens e mulheres. Testamos a plotagem de dois padrões de gráficos para escolher o que melhor se adequa.<br>
O gráfico de barras horizontais foi escolhido.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Codificada Sexo Sinistro', hue='cluster', data=df, palette='viridis')
plt.title('Relação entre Homens e Mulheres por cluster')
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.xticks(ticks=range(2), labels=['Mulheres', 'Homens'])
plt.legend(title='cluster')
plt.show()

In [ ]:
# Primeiro, calcule as contagens por gênero e cluster
contagens = df.groupby(['Codificada Sexo Sinistro', 'cluster']).size().unstack()

# Plote o gráfico de barras empilhadas
ax = contagens.plot(kind='barh', stacked=True, colormap='viridis', figsize=(10, 6))

# Personalize o gráfico
plt.title('Relação entre Homens e Mulheres por cluster')
plt.xlabel('Quantidade')
plt.ylabel('Gênero')
plt.yticks(ticks=range(2), labels=['Mulheres', 'Homens'])
plt.legend(title='cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

#### Relação entre Faixas Etárias por cluster
Esse gráfico mostra a distribuição das faixas etárias dentro de cada cluster, permitindo visualizar como diferentes<br>
grupos de idade estão distribuídos entre os clusters.

In [ ]:
# Contar a quantidade de ocorrências para cada combinação de faixa etária e cluster
contagem_df = df.groupby(['Codificada Faixa-Etária Nova Sinistro', 'cluster']).size().unstack(fill_value=0)

# Plotar gráfico de barras acumuladas
plt.figure(figsize=(10, 6))
contagem_df.plot(kind='bar', stacked=True, colormap='viridis', edgecolor='none')
plt.title('Relação entre Faixas Etárias por cluster')
plt.xlabel('Faixa Etária')
plt.ylabel('Quantidade')
plt.xticks(ticks=range(len(contagem_df.index)), labels=['0-18','19-23','24-28','29-33','34-38','39-43','44-48','49-53','54-58','58 ou mais'], rotation=45)
plt.legend(title='cluster')
plt.show()

#### Relação entre Valor Pago por cluster Mensalmente
Esse gráfico mostra a distribuição do valor pago por mês para cada cluster, permitindo comparar os gastos médios<br>
ou totais entre os diferentes grupos formados pelo K-means.<br>
(Ainda estamos analisando a viabilidade desse gráfico para o projeto)

In [ ]:
def tratar_outliers_por_cluster(df, coluna):
    # DataFrame para armazenar dados limpos
    df_limpo = pd.DataFrame()

    for cluster in df['cluster'].unique():
        # Selecionar dados para o cluster
        dados_cluster = df[df['cluster'] == cluster]
        
        # Calcular o IQR
        Q1 = dados_cluster[coluna].quantile(0.25)
        Q3 = dados_cluster[coluna].quantile(0.75)
        IQR = Q3 - Q1
        
        # Definir limites para outliers
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        # Filtrar dados sem outliers
        dados_limpos = dados_cluster[(dados_cluster[coluna] >= limite_inferior) & (dados_cluster[coluna] <= limite_superior)]
        
        # Adicionar dados limpos ao DataFrame final
        df_limpo = pd.concat([df_limpo, dados_limpos])
    
    return df_limpo

# Tratar outliers
dados_normalizado = tratar_outliers_por_cluster(df, 'Valor Pago Sinistro')

# Criar gráfico de caixa
plt.figure(figsize=(10, 6))
sns.boxplot(data=dados_normalizado, x='cluster', y='Valor Pago Sinistro', hue='cluster', palette='viridis', legend=False)

# Adicionar título e rótulos aos eixos
plt.title('Distribuição do Valor Pago Sinistro por cluster (Sem Outliers)')
plt.xlabel('cluster')
plt.ylabel('Valor Pago Sinistro')

# Exibir o gráfico
plt.show()

Identificação dos dados de um cluster

In [ ]:
cluster_1_data = df[df['cluster'] == 1]

cluster_1_data

Verifica a correlação entre _features_

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", linewidths=0.5)

plt.show()

---
## PCA
O método PCA é uma técnica de redução de dimensionalidade usada para simplificar grandes conjuntos de dados,<br>
preservando o máximo de variância possível. Utilizamos ele para obter uma melhor visualização dos dados.

### Visualização 2D

In [ ]:
pca_2d = PCA(n_components=2)
dados_treino_pca = pca_2d.fit_transform(df)
dados_teste_pca = pca_2d.transform(df)
dados_teste_pca_df = pd.DataFrame(dados_teste_pca, columns=['PC1', 'PC2'])
dados_teste_pca_df['cluster'] = df['cluster']

plt.figure(figsize=(10, 6))
sns.scatterplot(data=dados_teste_pca_df, x='PC1', y='PC2', hue=df['cluster'], palette='viridis', s=60)
plt.title('clusters previstos pelo K-Means (2D PCA)')
plt.show()

### Visualização 3D

In [ ]:
pca_3d = PCA(n_components=3)
dados_teste_pca = pca_3d.fit_transform(df)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dados_teste_pca[:, 0], dados_teste_pca[:, 1], dados_teste_pca[:, 2], c=df['cluster'], cmap='viridis')

ax.set_title("clusters previstos pelo K-Means (3D PCA)")
ax.set_xlabel("Componente Principal 1")
ax.set_ylabel("Componente Principal 2")
ax.set_zlabel("Componente Principal 3")

plt.show()

# Cálculo de métricas

In [ ]:
# Inicializando o modelo K-Means com 7 clusters
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300, random_state=42)
kmeans.fit(df)

# Adicionando rótulos ao DataFrame
df['cluster'] = kmeans.labels_

# Calculando o Silhouette Score
silhouette_avg = silhouette_score(df, df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

# Calculando o Davies-Bouldin Index
dbi = davies_bouldin_score(df, df['cluster'])
print(f"Davies-Bouldin Index: {dbi}")

# Calculando o Calinski-Harabasz Index
chi = calinski_harabasz_score(df, df['cluster'])
print(f"Calinski-Harabasz Index: {chi}")
